In [1]:
import numpy as np

In [32]:
class NotAFeatureError(Exception):
    pass

class FeatureBank:
    def __init__(self):
        self._feat_list = []
        
        self.n_feats = []
        
        self._eq_idx = []
    
    def __add__(self, other):
        if isinstance(other, Feature):
            self._feat_list.append(DeferredFeatureResult(other, ...))
        elif isinstance(other, DeferredFeatureResult):
            if other in self:
                self._feat_list.append(DeferredFeatureResult(self._feat_list[self._eq_idx].parent, other.index))
            else:
                self._feat_list.append(other)
        else:
            raise NotAFeatureError(f'Cannot add a non-Feature to the FeatureBank')
    
    def compute(self, x):
        for ft in self._feat_list:
            # need the if statement to deal with ellipsis indexes
            if ft.n == -1:
                if x.ndim > 1:
                    ft.n = x.shape[1]  # 1 should always be the number of axes
                else:
                    ft.n = 1
            self.n_feats.append(ft.n)
        
        feats = np.zeros((x.shape[0], np.sum(self.n_feats)))
        
        idx = 0
        
        for i, dfr in enumerate(self._feat_list):
            dfr._compute(x=x)
            
            if self.n_feats[i] > 1:
                feats[:, idx:idx+self.n_feats[i]] = dfr.get_result()
            else:
                feats[:, idx] = dfr.get_result()
            
            idx += self.n_feats[i]
        
        return feats
            
            
    def __contains__(self, item):
        isin = False
        
        if isinstance(item, (Feature, DeferredFeatureResult)):
            for i, dfr in enumerate(self._feat_list):
                comp = item == dfr
                isin |= comp
                
                # save the location of the last equivalent item
                if comp:
                    self._eq_idx = i
        
        return isin
        
class DeferredFeatureResult:
    def __init__(self, parent, index):
        self.parent = parent
        self.index = index
        self._compute = self.parent._compute  # shortcut the compute name
        
        if hasattr(index, "__len__"):
            self.n = len(index)
        elif isinstance(index, type(Ellipsis)):
            self.n = -1
        else:
            self.n = 1
    
    def get_result(self):
        return self.parent.result[:, self.index]
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return other.parent._eq_params == self.parent._eq_params
        elif isinstance(other, Feature):
            return other._eq_params == self.parent._eq_params
        else:
            return False
    
    
class Feature:
    def __init__(self, eq_params={}):
        self._x = None
        self.result = None
        
        self._xyz_map = {'x': 0, 'y': 1, 'z': 2}
        self._index = []
        
        self._eq_params = eq_params  # needs to be overwritten to properly check equivalence
        
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return other._eq_params == self._eq_params
        elif isinstance(other, DeferredFeatureResult):
            return other.parent._eq_params == self._eq_params
        else:
            return False
        
    def __getitem__(self, key):
        if isinstance(key, (str, int)):
            if key in self._xyz_map:
                index = self._xyz_map[key]
            else:
                index = key
        else:
            if isinstance(key[0], str):
                index = [self._xyz_map[i] for i in key]
            else:
                index = key
        
        return DeferredFeatureResult(self, index)
    
    def _compute(self, x=None):
        if x is not None:
            self._x = x
        else:
            if self._x is None:
                raise ValueError('Must provide signal to analyze')
        
        # if the result is already defined, don't need to compute again
        if self.result is not None:
            return
        
    
    def compute(self, x):
        self._compute(x=x)
        return self.result
        
        
class F1(Feature):
    def __init__(self, param1=5.0):
        super().__init__(eq_params={'param1': param1})
        
        self.param1 = param1
        
    def _compute(self, x=None):
        super()._compute(x=x)
        
        self.result = self._x + self.param1

class F2(Feature):
    def __init__(self, factor=2.5):
        super().__init__(eq_params={'factor': factor})
        
        self.factor = factor
        
    def _compute(self, x=None):
        super()._compute(x=x)
        
        self.result = -self._x * self.factor

In [33]:
f1a = F1(param1=5.0)
f1b = F1(param1=10.0)

f2a = F2(factor=2.5)
f2b = F2(factor=3.25)

fb = FeatureBank()
# create the list of features
fb + f2a[0]
fb + f1a
fb + f1b[2]
fb + f2b[[0, 2]]

In [34]:
x = np.arange(15).reshape((5, 3))

In [35]:
fb.compute(x)

array([[  0.  ,   5.  ,   6.  ,   7.  ,  12.  ,   0.  ,  -6.5 ],
       [ -7.5 ,   8.  ,   9.  ,  10.  ,  15.  ,  -9.75, -16.25],
       [-15.  ,  11.  ,  12.  ,  13.  ,  18.  , -19.5 , -26.  ],
       [-22.5 ,  14.  ,  15.  ,  16.  ,  21.  , -29.25, -35.75],
       [-30.  ,  17.  ,  18.  ,  19.  ,  24.  , -39.  , -45.5 ]])

In [36]:
print(f1a == f1b)
print(f2a == f2b)

False
False


In [40]:
bank = FeatureBank()

bank + F1(param1=5.0)[[0, 2]]
bank + F1(param1=10.0)[1]
bank + F2()
bank + F1(param1=5.0)[1]

In [42]:
bank._feat_list[-1].parent is bank._feat_list[0].parent

True